### Development notebook to measure change in a cycle network over time

- Select random LSOA
- Get appropriate bounding boxes
- Get OSM data 2016 and 2021
- Test following options
    - OD2NET
    - GraphScope (https://stackoverflow.com/questions/53044151/python-computing-weighted-betweenness-centrality-for-nodes-of-a-huge-graph)
    - Graph tool
    - Efficient all-pair SP aglorithms?
    - Edge centrality heuristics (e.g., sampling?)
- OD2NET
    - Output as .osm.pbf file
    - Instantiate OD2NET to get LTS measures and all pairs centrality
    - 
- GraphScope
    - Convert networkx to graphscope object
    - Run centrality function
    - Time comparison
    - Accuracy comparios
- Graph tool
    - Same as above
- Read in network to python to perform 


NOTE:
Potential additional approach to speed up - extract full UK network 2016 vs 2021 and measure LTS etc. Then clip network to bounding boxes for each LSOA. Currently we dynamically pull network for LSOA, which means the same area will be extracted and have LTS measured multiple times. Not neccessarily incorrect approach, as scaling must also be considered (e.g., if pulling network scales non-linearly it may not be more efficient). Worth analysing two approach though once above resolved.

Another possible approach. Tolerate inefficiency and use parallel computing.

In [1]:
import graph_tool.all as gt
import osmnx as ox
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from shapely.geometry import box, Polygon, MultiPolygon
from shapely.ops import transform
from functools import partial
import pyproj
import OSM2AT
import networkx as nx
import pickle 
import sys
import statistics
sys.stdout = open('term_output.txt', 'w')

#Cycle network
impute_method = 'mode-rule'
mlp_train_params = {
    'hidden_layer' : 100,
    'n_epochs' : 50,
    'batch_size' : 10
}
lts_method = 'ottawa'
self_learn_k = 5
pull_method = 'bb'
place = None

bbx_expansion = 0.5

#Below function from ChatGPT
#Get expanded network - method 1 km buffer
def expand_bbox(original_bbox, expansion_distance_km=5):
    # Create a Shapely geometry object for the original bounding box
    original_geometry = box(*original_bbox)
    # Define a function to project the geometry to a new coordinate reference system
    project = partial(
        pyproj.transform,
        pyproj.Proj(init='epsg:4326'),  # WGS 84 coordinate reference system
        pyproj.Proj(proj='utm', zone=33, ellps='WGS84')  # Example: UTM Zone 33
    )
    # Project the original geometry to the new coordinate reference system
    projected_geometry = transform(project, original_geometry)
    # Calculate the expansion distance in the projected coordinate system
    expansion_distance_meters = expansion_distance_km * 1000
    # Expand the geometry by the specified distance
    expanded_geometry = projected_geometry.buffer(expansion_distance_meters)
    # Project the expanded geometry back to the original coordinate reference system
    expanded_geometry = transform(partial(pyproj.transform, pyproj.Proj(proj='utm', zone=33, ellps='WGS84'), pyproj.Proj(init='epsg:4326')), expanded_geometry)
    # Get the coordinates of the expanded bounding box
    expanded_bbox = expanded_geometry.bounds
    return expanded_bbox, expanded_geometry

def create_bounding_box(geometry1, geometry2):

    # Calculate the union of all polygons in each multipolygon
    union_geometry1 = geometry1.convex_hull
    union_geometry2 = geometry2.convex_hull
    # Calculate the union of the convex hulls of the two multipolygons
    union_geometry = union_geometry1.union(union_geometry2)
    # Get the bounding box of the union geometry
    bounding_box = union_geometry.bounds
    return bounding_box

def get_centrality(edges):

    edge_list = []
    for i,e in edges.iterrows():
        edge_list.append(tuple([str(i[0]),str(i[1]),e['length'],e['LTS']]))
        
    g = gt.Graph()

    elength = g.new_ep("float")
    elts = g.new_ep("int")

    g.add_edge_list(edge_list, hashed = True,eprops=[elength, elts])

    vp, ep = gt.betweenness(g,weight=elength)

    lts0 = []
    lts1 = []
    lts2 = []
    lts3 = []
    lts4 = []

    for e in g.edges():
        if elts[e] == 0:
            lts0.append(ep[e])
        elif elts[e] == 1:
            lts1.append(ep[e])
        elif elts[e] == 2:
            lts2.append(ep[e])
        elif elts[e] == 3:
            lts3.append(ep[e])
        elif elts[e] == 4:
            lts4.append(ep[e])

    return lts0,lts1,lts2,lts3,lts4

/home/chris/miniconda3/envs/gt/lib/python3.12/site-packages/OSM2AT/get_at_networks.py:31: FutureWarning: The `utils.config` function is deprecated and will be removed in the v2.0.0 release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(use_cache=True, log_console=True, useful_tags_way=utw)


In [2]:
# Import MSOA lookup
msoas = gpd.read_file('data/MSOA_EngWal_Dec_2011_Generalised_ClippedEW_0/Middle_Layer_Super_Output_Areas_December_2011_Generalised_Clipped_Boundaries_in_England_and_Wales.shp').to_crs(4326).set_index('msoa11cd')

# Import MSOA 2011 OD data
od_data = pd.read_parquet('data/od_2011.parquet')

#Import LSOAs
lsoas = gpd.read_file('data/LSOA_2011_Boundaries_Super_Generalised_Clipped_BSC_EW_V4_6029841263726194941.gpkg').to_crs(4326)
lsoas = pd.concat([lsoas, lsoas.bounds], axis=1)

#Import lsoa to msoa look up
lookup = pd.read_csv('data/PCD11_OA11_LSOA11_MSOA11_LAD11_EW_LU_aligned_v2.csv')

/tmp/ipykernel_202965/2575286281.py:12: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  lookup = pd.read_csv('data/PCD11_OA11_LSOA11_MSOA11_LAD11_EW_LU_aligned_v2.csv')


In [3]:
#Select different time periods to be studied

settings_2016 = '[out:json][timeout:90][date:"2016-06-01T01:00:00Z"]'
settings_2021 = '[out:json][timeout:90][date:"2021-06-01T01:00:00Z"]'

In [4]:
# Use nested dictionaries to capture networks at different levels and time periods

networks = {}
networks['LSOA'] = {}
networks['LSOA']['2016'] = {}
networks['LSOA']['2021'] = {}
networks['LSOA_Expanded'] = {}
networks['LSOA_Expanded']['2016'] = {}
networks['LSOA_Expanded']['2021'] = {}
networks['LSOA_OD'] = {}
networks['LSOA_OD']['2016'] = {}
networks['LSOA_OD']['2021'] = {}
networks['LSOA_OD_No_LTS4'] = {}
networks['LSOA_OD_No_LTS4']['2016'] = {}
networks['LSOA_OD_No_LTS4']['2021'] = {}
networks['LSOA_OD_LTS3+'] = {}
networks['LSOA_OD_LTS3+']['2016'] = {}
networks['LSOA_OD_LTS3+']['2021'] = {}

metrics = {}

york_model = pd.read_csv('data/LSOA_york_model.csv')
york_lsoas = list(york_model['LSOA_code'])

# lsoa_id = york_lsoas[1]
# #lsoa = lsoas[lsoas['LSOA11CD'] == test_lsoa['LSOA11CD'].values[0]]
# lsoa_lookup = lookup[lookup['LSOA11CD'] == lsoa_id][:1]
# lsoa = lsoas[lsoas['LSOA11CD'] == lsoa_lookup['LSOA11CD'].values[0]]

# metrics[lsoa_id] = {}
# metrics[lsoa_id]['LSOA'] = {}
# metrics[lsoa_id]['LSOA_Expanded'] = {}
# metrics[lsoa_id]['LSOA_OD'] = {}

In [5]:
#Iterate through York OAs
for lsoa_id in york_lsoas:
    
    #lsoa = lsoas[lsoas['LSOA11CD'] == test_lsoa['LSOA11CD'].values[0]]
    lsoa_lookup = lookup[lookup['LSOA11CD'] == lsoa_id][:1]
    lsoa = lsoas[lsoas['LSOA11CD'] == lsoa_lookup['LSOA11CD'].values[0]]

    metrics[lsoa_id] = {}
    metrics[lsoa_id]['LSOA'] = {}
    metrics[lsoa_id]['LSOA_Expanded'] = {}
    metrics[lsoa_id]['LSOA_OD'] = {}

    #Part 1 - Extract networks for different periods and different areas of study

    #Get LSOA networks
    ox.settings.overpass_settings = settings_2016
    G = ox.graph_from_bbox(lsoa['maxy'],lsoa['miny'],lsoa['minx'],lsoa['maxx'],network_type = 'bike',retain_all=True,simplify=False)
    G,edges = OSM2AT.measure_LTS_from_network(G,impute_method,mlp_train_params,lts_method,self_learn_k)

    networks['LSOA']['2016']['Graph'] = G
    networks['LSOA']['2016']['Edges'] = edges

    ox.settings.overpass_settings = settings_2021
    G = ox.graph_from_bbox(lsoa['maxy'],lsoa['miny'],lsoa['minx'],lsoa['maxx'],network_type = 'bike',retain_all=True,simplify=False)
    G,edges = OSM2AT.measure_LTS_from_network(G,impute_method,mlp_train_params,lts_method,self_learn_k)

    networks['LSOA']['2021']['Graph'] = G
    networks['LSOA']['2021']['Edges'] = edges

    #Get expanded LSOA networks
    expanded_bbox, expanded_geometry = expand_bbox((lsoa['minx'], lsoa['miny'], lsoa['maxx'], lsoa['maxy']), expansion_distance_km=bbx_expansion)
    box_expanded = box(expanded_bbox[2],expanded_bbox[1],expanded_bbox[0],expanded_bbox[3])

    ox.settings.overpass_settings = settings_2016
    G = ox.graph_from_bbox(expanded_bbox[3],expanded_bbox[1],expanded_bbox[2],expanded_bbox[0],network_type = 'bike',retain_all=True,simplify=False)
    G,edges = OSM2AT.measure_LTS_from_network(G,impute_method,mlp_train_params,lts_method,self_learn_k)

    networks['LSOA_Expanded']['2016']['Graph'] = G
    networks['LSOA_Expanded']['2016']['Edges'] = edges

    ox.settings.overpass_settings = settings_2021
    G = ox.graph_from_bbox(expanded_bbox[3],expanded_bbox[1],expanded_bbox[2],expanded_bbox[0],network_type = 'bike',retain_all=True,simplify=False)
    G,edges = OSM2AT.measure_LTS_from_network(G,impute_method,mlp_train_params,lts_method,self_learn_k)

    networks['LSOA_Expanded']['2021']['Graph'] = G
    networks['LSOA_Expanded']['2021']['Edges'] = edges

    #Get OD Expanded Network

    bike_ods = od_data[(od_data['geo_code1'] == lsoa_lookup['MSOA11CD'].values[0]) & (od_data['bicycle'] > 0)][['geo_code2','bicycle']].set_index('geo_code2')
    bike_ods['geometry'] = msoas['geometry']
    bike_ods = bike_ods.dropna()

    origin_geom = lsoa['geometry'].values[0]
    destination_geom = msoas.loc[bike_ods['bicycle'].idxmax()]['geometry']

    bounding_box_od = create_bounding_box(origin_geom, destination_geom)

    ox.settings.overpass_settings = settings_2016
    G = ox.graph_from_bbox(bounding_box_od[3],bounding_box_od[1],bounding_box_od[2],bounding_box_od[0],network_type = 'bike',retain_all=True,simplify=False)
    G,edges = OSM2AT.measure_LTS_from_network(G,impute_method,mlp_train_params,lts_method,self_learn_k)

    networks['LSOA_OD']['2016']['Graph'] = G
    networks['LSOA_OD']['2016']['Edges'] = edges

    ox.settings.overpass_settings = settings_2021
    G = ox.graph_from_bbox(bounding_box_od[3],bounding_box_od[1],bounding_box_od[2],bounding_box_od[0],network_type = 'bike',retain_all=True,simplify=False)
    G,edges = OSM2AT.measure_LTS_from_network(G,impute_method,mlp_train_params,lts_method,self_learn_k)
    networks['LSOA_OD']['2021']['Graph'] = G
    networks['LSOA_OD']['2021']['Edges'] = edges

    #Part 2 - Update networks to remove dangerous edges (e.g., LTS 4)

    #Remove LTS4
    G = networks['LSOA_OD']['2016']['Graph'].copy()
    edges = networks['LSOA_OD']['2016']['Edges'].copy()

    for i,r in networks['LSOA_OD']['2016']['Edges'][networks['LSOA_OD']['2016']['Edges']['LTS'] == 4].iterrows():
        G.remove_edge(i[0],i[1],0)
        edges = edges.drop(i)

    networks['LSOA_OD_No_LTS4']['2016']['Graph'] = G
    networks['LSOA_OD_No_LTS4']['2016']['Edges'] = edges

    #2021
    G = networks['LSOA_OD']['2021']['Graph'].copy()
    edges = networks['LSOA_OD']['2021']['Edges'].copy()

    for i,r in networks['LSOA_OD']['2021']['Edges'][networks['LSOA_OD']['2021']['Edges']['LTS'] == 4].iterrows():
        G.remove_edge(i[0],i[1],0)
        edges = edges.drop(i)

    networks['LSOA_OD_No_LTS4']['2021']['Graph'] = G
    networks['LSOA_OD_No_LTS4']['2021']['Edges'] = edges

    # Increase access cost of LTS 3
    #2016
    G = networks['LSOA_OD_No_LTS4']['2016']['Graph'].copy()
    edges = networks['LSOA_OD_No_LTS4']['2016']['Edges'].copy()

    for i,r in networks['LSOA_OD_No_LTS4']['2016']['Edges'][networks['LSOA_OD_No_LTS4']['2016']['Edges']['LTS'] == 3].iterrows():
        new_edge_length = (G[i[0]][i[1]][0]['length'] * 1.5)
        G[i[0]][i[1]][0]['length'] = new_edge_length
        edges.at[i,'length'] = new_edge_length
        
    networks['LSOA_OD_LTS3+']['2016']['Graph'] = G
    networks['LSOA_OD_LTS3+']['2016']['Edges'] = edges

    #2021
    G = networks['LSOA_OD_No_LTS4']['2021']['Graph'].copy()
    edges = networks['LSOA_OD_No_LTS4']['2021']['Edges'].copy()

    for i,r in networks['LSOA_OD_No_LTS4']['2021']['Edges'][networks['LSOA_OD_No_LTS4']['2021']['Edges']['LTS'] == 3].iterrows():
        new_edge_length = (G[i[0]][i[1]][0]['length'] * 1.5)
        G[i[0]][i[1]][0]['length'] = new_edge_length
        edges.at[i,'length'] = new_edge_length
        
    networks['LSOA_OD_LTS3+']['2021']['Graph'] = G
    networks['LSOA_OD_LTS3+']['2021']['Edges'] = edges

    #Part 3 measure change infrastructure over time at different network levels - percentage change in LTS 1,2 3 etc in terms of number of edges and KM proportion in network

    #Increase in cycleway infrastucuture
    for level in ['LSOA','LSOA_Expanded','LSOA_OD']:
        grouped_data_2016 = networks[level]['2016']['Edges'].groupby('highway')['length'].sum()
        grouped_data_2021 = networks[level]['2021']['Edges'].groupby('highway')['length'].sum()

        # Calculate percentages
        percentage_2016 = (grouped_data_2016 / grouped_data_2016.sum()) * 100
        percentage_2021 = (grouped_data_2021 / grouped_data_2021.sum()) * 100

        if 'cycleway' in grouped_data_2016.index:
            cycleways16 = grouped_data_2016['cycleway']
        else:
            cycleways16 = 0

        if 'cycleway' in grouped_data_2021.index:
            cycleways21 = grouped_data_2021['cycleway']
        else:
            cycleways21 = 0

        if 'cycleway' in percentage_2016.index:
            cycleways16_pcnt = percentage_2016['cycleway']
        else:
            cycleways16_pcnt = 0

        if 'cycleway' in percentage_2021.index:
            cycleways21_pcnt = percentage_2021['cycleway']
        else:
            cycleways21_pcnt = 0

        metrics[lsoa_id][level]['Increase in Cycleways KM'] = cycleways21 - cycleways16
        metrics[lsoa_id][level]['Increase in Cycleways %'] = cycleways21_pcnt - cycleways16_pcnt
        if cycleways16 > 0:
            metrics[lsoa_id][level]['% Increase in Cycleways KM'] = (cycleways21 - cycleways16) / cycleways16
            metrics[lsoa_id][level]['% Increase in Cycleways %'] = (cycleways21_pcnt - cycleways16_pcnt) / cycleways16_pcnt
        else:
            metrics[lsoa_id][level]['% Increase in Cycleways KM'] = None
            metrics[lsoa_id][level]['% Increase in Cycleways %'] = None
        
    for level in ['LSOA','LSOA_Expanded','LSOA_OD']:
        grouped_data_2016 = networks[level]['2016']['Edges'].groupby('LTS')['length'].sum()
        grouped_data_2021 = networks[level]['2021']['Edges'].groupby('LTS')['length'].sum()

        if 1 not in grouped_data_2016.index:
            grouped_data_2016[1] = 0
        if 2 not in grouped_data_2016.index:
            grouped_data_2016[2] = 0
        if 1 not in grouped_data_2021.index:
            grouped_data_2021[1] = 0
        if 2 not in grouped_data_2016.index:
            grouped_data_2021[2] = 0
        
        # Calculate percentages
        percentage_2016 = (grouped_data_2016 / grouped_data_2016.sum()) * 100
        percentage_2021 = (grouped_data_2021 / grouped_data_2021.sum()) * 100

        # Increase in LTS 1 and 2

        lts_1_2_2016 = grouped_data_2016.loc[1] + grouped_data_2016.loc[2]
        lts_1_2_2021 = grouped_data_2021.loc[1] + grouped_data_2021.loc[2]

        lts_1_2_2016_pcnt = percentage_2016.loc[1] + percentage_2016.loc[2]
        lts_1_2_2021_pcnt = percentage_2021.loc[1] + percentage_2021.loc[2]

        metrics[lsoa_id][level]['Increase in LTS 1 and 2 KM'] = lts_1_2_2021 - lts_1_2_2016
        metrics[lsoa_id][level]['Increase in LTS 1 and 2 %'] = lts_1_2_2021_pcnt - lts_1_2_2016_pcnt
        if lts_1_2_2016 > 0:
            metrics[lsoa_id][level]['% Increase in LTS 1 and 2 KM'] = (lts_1_2_2021 - lts_1_2_2016) / lts_1_2_2016
            metrics[lsoa_id][level]['% Increase in LTS 1 and 2 %'] = (lts_1_2_2021_pcnt - lts_1_2_2016_pcnt) / lts_1_2_2016_pcnt
        else:
            metrics[lsoa_id][level]['% Increase in LTS 1 and 2 KM'] = None
            metrics[lsoa_id][level]['% Increase in LTS 1 and 2 %'] = None
    
    #Part 4 - get routing metrics e.g., change in dangerous path penalty etc
    
    # Get LSOA centroid
    o_lon = origin_geom.centroid.x
    o_lat = origin_geom.centroid.y

    # Get destination lsoa centroid
    d_lon = destination_geom.centroid.x
    d_lat = destination_geom.centroid.y

    #Get best paths
    path_2016 = ox.routing.shortest_path(networks['LSOA_OD']['2016']['Graph'], ox.nearest_nodes(networks['LSOA_OD']['2016']['Graph'],o_lon,o_lat), ox.nearest_nodes(networks['LSOA_OD']['2016']['Graph'],d_lon,d_lat), weight='length')
    paths_2021 = ox.routing.shortest_path(networks['LSOA_OD']['2021']['Graph'], ox.nearest_nodes(networks['LSOA_OD']['2021']['Graph'],o_lon,o_lat), ox.nearest_nodes(networks['LSOA_OD']['2021']['Graph'],d_lon,d_lat), weight='length')
    
    #Part 4.1 - change in proportion of best path which is LTS 1/2
    
    # Get KM by LTS
    path_results_list = []

    lts0 = 0
    lts1 = 0
    lts2 = 0
    lts3 = 0
    lts4 = 0

    if (path_2016 is None) or (paths_2021 is None):
        metrics[lsoa_id]['LSOA_OD']['Best path LTS1/2 Proportion Increase'] = None
        metrics[lsoa_id]['LSOA_OD']['Best path LTS1/2 Proportion Increase Percent'] = None 
    else:
        u = path_2016[0]
        for v in path_2016[1:]:
            edge = networks['LSOA_OD']['2016']['Edges'].loc[u,v,0][['LTS','length']]
            if edge['LTS'] == 0:
                lts0 += edge['length']
            if edge['LTS'] == 1:
                lts1 += edge['length']
            if edge['LTS'] == 2:
                lts2 += edge['length']
            if edge['LTS'] == 3:
                lts3 += edge['length']
            if edge['LTS'] == 4:
                lts4 += edge['length']
            u = v
        path_result_append = {}
        path_result_append['year'] = '2016'
        path_result_append['lts0'] = lts0
        path_result_append['lts1'] = lts1
        path_result_append['lts2'] = lts2
        path_result_append['lts3'] = lts3
        path_result_append['lts4'] = lts4
        path_results_list.append(path_result_append)


        lts0 = 0
        lts1 = 0
        lts2 = 0
        lts3 = 0
        lts4 = 0

        u = paths_2021[0]
        for v in paths_2021[1:]:
            edge = networks['LSOA_OD']['2021']['Edges'].loc[u,v,0][['LTS','length']]
            if edge['LTS'] == 0:
                lts0 += edge['length']
            if edge['LTS'] == 1:
                lts1 += edge['length']
            if edge['LTS'] == 2:
                lts2 += edge['length']
            if edge['LTS'] == 3:
                lts3 += edge['length']
            if edge['LTS'] == 4:
                lts4 += edge['length']
            u = v
        path_result_append = {}
        path_result_append['year'] = '2021'
        path_result_append['lts0'] = lts0
        path_result_append['lts1'] = lts1
        path_result_append['lts2'] = lts2
        path_result_append['lts3'] = lts3
        path_result_append['lts4'] = lts4
        path_results_list.append(path_result_append)

        path_results_od = pd.DataFrame(path_results_list).set_index('year')

        # Create a new DataFrame to store the percentages
        lts_percentages = path_results_od.copy()
        # Divide each element in the original DataFrame by the corresponding row sum
        lts_percentages = lts_percentages.div(path_results_od.sum(axis=1), axis=0) * 100

        lts_12_2016 = lts_percentages.loc['2016']['lts1'] + lts_percentages.loc['2016']['lts2']
        lts_12_2021 = lts_percentages.loc['2021']['lts1'] + lts_percentages.loc['2021']['lts2']

        proportion_lts_12_increase = lts_12_2021 - lts_12_2016
        percentage_increase = (lts_12_2021 - lts_12_2016) / lts_12_2016
        metrics[lsoa_id]['LSOA_OD']['Best path LTS1/2 Proportion Increase'] = proportion_lts_12_increase
        metrics[lsoa_id]['LSOA_OD']['Best path LTS1/2 Proportion Increase Percent'] = percentage_increase

    #Part 4.2 - Dangerous road penalty

    #Get best paths
    path_2016 = ox.routing.shortest_path(networks['LSOA_OD_No_LTS4']['2016']['Graph'], ox.nearest_nodes(networks['LSOA_OD_No_LTS4']['2016']['Graph'],o_lon,o_lat), ox.nearest_nodes(networks['LSOA_OD_No_LTS4']['2016']['Graph'],d_lon,d_lat), weight='length')
    paths_2021 = ox.routing.shortest_path(networks['LSOA_OD_No_LTS4']['2021']['Graph'], ox.nearest_nodes(networks['LSOA_OD_No_LTS4']['2021']['Graph'],o_lon,o_lat), ox.nearest_nodes(networks['LSOA_OD_No_LTS4']['2021']['Graph'],d_lon,d_lat), weight='length')

    # Get KM by LTS
    path_results_list = []

    lts0 = 0
    lts1 = 0
    lts2 = 0
    lts3 = 0
    lts4 = 0
    if (path_2016 is None) or (paths_2021 is None):
        metrics[lsoa_id]['LSOA_OD']['LTS 3 Penalty Increase'] = None
        metrics[lsoa_id]['LSOA_OD']['LTS 3 Penalty Increase percent'] = None
        
    else:
        u = path_2016[0]
        for v in path_2016[1:]:
            edge = networks['LSOA_OD_No_LTS4']['2016']['Edges'].loc[u,v,0][['LTS','length']]
            if edge['LTS'] == 0:
                lts0 += edge['length']
            if edge['LTS'] == 1:
                lts1 += edge['length']
            if edge['LTS'] == 2:
                lts2 += edge['length']
            if edge['LTS'] == 3:
                lts3 += edge['length']
            if edge['LTS'] == 4:
                lts4 += edge['length']
            u = v
        path_result_append = {}
        path_result_append['year'] = '2016'
        path_result_append['lts0'] = lts0
        path_result_append['lts1'] = lts1
        path_result_append['lts2'] = lts2
        path_result_append['lts3'] = lts3
        path_result_append['lts4'] = lts4
        path_results_list.append(path_result_append)


        lts0 = 0
        lts1 = 0
        lts2 = 0
        lts3 = 0
        lts4 = 0

        u = paths_2021[0]
        for v in paths_2021[1:]:
            edge = networks['LSOA_OD_No_LTS4']['2021']['Edges'].loc[u,v,0][['LTS','length']]
            if edge['LTS'] == 0:
                lts0 += edge['length']
            if edge['LTS'] == 1:
                lts1 += edge['length']
            if edge['LTS'] == 2:
                lts2 += edge['length']
            if edge['LTS'] == 3:
                lts3 += edge['length']
            if edge['LTS'] == 4:
                lts4 += edge['length']
            u = v
        path_result_append = {}
        path_result_append['year'] = '2021'
        path_result_append['lts0'] = lts0
        path_result_append['lts1'] = lts1
        path_result_append['lts2'] = lts2
        path_result_append['lts3'] = lts3
        path_result_append['lts4'] = lts4
        path_results_list.append(path_result_append)

        path_results_od_no_lts4 = pd.DataFrame(path_results_list).set_index('year')

        penalties = path_results_od_no_lts4.sum(axis=1) - path_results_od.sum(axis=1)

        pen_proportions = penalties / path_results_od_no_lts4.sum(axis=1)

        dangerous_road_penalty_increase = penalties['2021'] - penalties['2016']
        dangerous_road_penalty_proportion_increase = pen_proportions['2021'] - pen_proportions['2016']

        metrics[lsoa_id]['LSOA_OD']['Dangerous road penalty increase'] = dangerous_road_penalty_increase
        metrics[lsoa_id]['LSOA_OD']['Dangerous road penalty increase percent'] = dangerous_road_penalty_proportion_increase

    #Get best paths
    path_2016 = ox.routing.shortest_path(networks['LSOA_OD_LTS3+']['2016']['Graph'], ox.nearest_nodes(networks['LSOA_OD_LTS3+']['2016']['Graph'],o_lon,o_lat), ox.nearest_nodes(networks['LSOA_OD_LTS3+']['2016']['Graph'],d_lon,d_lat), weight='length')
    paths_2021 = ox.routing.shortest_path(networks['LSOA_OD_LTS3+']['2021']['Graph'], ox.nearest_nodes(networks['LSOA_OD_LTS3+']['2021']['Graph'],o_lon,o_lat), ox.nearest_nodes(networks['LSOA_OD_LTS3+']['2021']['Graph'],d_lon,d_lat), weight='length')

    #Part 4.3 - LTS 3 penalty change over time

    # Get KM by LTS
    path_results_list = []

    lts0 = 0
    lts1 = 0
    lts2 = 0
    lts3 = 0
    lts4 = 0

    if (path_2016 is None) or (paths_2021 is None):
        
        metrics[lsoa_id]['LSOA_OD']['LTS 3 Penalty Increase'] = None
        metrics[lsoa_id]['LSOA_OD']['LTS 3 Penalty Increase percent'] = None

    else:
        u = path_2016[0]
        for v in path_2016[1:]:
            edge = networks['LSOA_OD_LTS3+']['2016']['Edges'].loc[u,v,0][['LTS','length']]
            if edge['LTS'] == 0:
                lts0 += edge['length']
            if edge['LTS'] == 1:
                lts1 += edge['length']
            if edge['LTS'] == 2:
                lts2 += edge['length']
            if edge['LTS'] == 3:
                lts3 += edge['length']
            if edge['LTS'] == 4:
                lts4 += edge['length']
            u = v
        path_result_append = {}
        path_result_append['year'] = '2016'
        path_result_append['lts0'] = lts0
        path_result_append['lts1'] = lts1
        path_result_append['lts2'] = lts2
        path_result_append['lts3'] = lts3
        path_result_append['lts4'] = lts4
        path_results_list.append(path_result_append)


        lts0 = 0
        lts1 = 0
        lts2 = 0
        lts3 = 0
        lts4 = 0

        u = paths_2021[0]
        for v in paths_2021[1:]:
            edge = networks['LSOA_OD_LTS3+']['2021']['Edges'].loc[u,v,0][['LTS','length']]
            if edge['LTS'] == 0:
                lts0 += edge['length']
            if edge['LTS'] == 1:
                lts1 += edge['length']
            if edge['LTS'] == 2:
                lts2 += edge['length']
            if edge['LTS'] == 3:
                lts3 += edge['length']
            if edge['LTS'] == 4:
                lts4 += edge['length']
            u = v
        path_result_append = {}
        path_result_append['year'] = '2021'
        path_result_append['lts0'] = lts0
        path_result_append['lts1'] = lts1
        path_result_append['lts2'] = lts2
        path_result_append['lts3'] = lts3
        path_result_append['lts4'] = lts4
        path_results_list.append(path_result_append)

        path_results_od_lts3_plus = pd.DataFrame(path_results_list).set_index('year')

        lts3_2016_pen = path_results_od_lts3_plus.sum(axis=1)['2016'] - path_results_od_no_lts4.sum(axis=1)['2016']
        lts3_2021_pen = path_results_od_lts3_plus.sum(axis=1)['2021'] - path_results_od_no_lts4.sum(axis=1)['2021']

        lts3_penality_increase = lts3_2021_pen - lts3_2016_pen
        lts3_penality_increase_percent = (lts3_2021_pen - lts3_2016_pen)/lts3_2016_pen

        metrics[lsoa_id]['LSOA_OD']['LTS 3 Penalty Increase'] = lts3_penality_increase
        metrics[lsoa_id]['LSOA_OD']['LTS 3 Penalty Increase percent'] = lts3_penality_increase_percent
        
    #Part 5 - change in centality over time    
    
    for level in ['LSOA','LSOA_Expanded','LSOA_OD']:
        lts0_16,lts1_16,lts2_16,lts3_16,lts4_16 = get_centrality(networks[level]['2016']['Edges'])
        lts0_21,lts1_21,lts2_21,lts3_21,lts4_21 = get_centrality(networks[level]['2021']['Edges'])
        
        if len(lts1_16) > 0:
            cent_lt1_16 = statistics.mean(lts1_16)
        else:
            cent_lt1_16 = 0
            
        if len(lts1_21) > 0:
            cent_lt1_21 = statistics.mean(lts1_21)
        else:
            cent_lt1_21 = 0
            
        if len(lts2_16) > 0:
            cent_lt2_16 = statistics.mean(lts2_16)
        else:
            cent_lt2_16 = 0
            
        if len(lts2_21) > 0:
            cent_lt2_21 = statistics.mean(lts2_21)
        else:
            cent_lt2_21 = 0
            
        increase_lts1_centrality = cent_lt1_21 - cent_lt1_16
        increase_lts1_centrality_pcnt = (cent_lt1_21 - cent_lt1_16) / cent_lt1_16

        increase_lts2_centrality = cent_lt2_21 - cent_lt2_16
        increase_lts2_centrality_pcnt = (cent_lt2_21 - cent_lt2_16) / cent_lt2_16

        metrics[lsoa_id][level]['Increase in LTS 1 Centrality'] = increase_lts1_centrality
        metrics[lsoa_id][level]['Increase in LTS 1 Centrality Percent'] = increase_lts1_centrality_pcnt
        metrics[lsoa_id][level]['Increase in LTS 2 Centrality'] = increase_lts2_centrality
        metrics[lsoa_id][level]['Increase in LTS 2 Centrality Percent'] = increase_lts2_centrality_pcnt
        
    with open('lsoa_metrics_new.pkl', 'wb') as f:
        pickle.dump(metrics, f)

/tmp/ipykernel_202965/722371406.py:14: FutureWarning: The `north`, `south`, `east`, and `west` parameters are deprecated and will be removed in the v2.0.0 release. Use the `bbox` parameter instead.
  G = ox.graph_from_bbox(lsoa['maxy'],lsoa['miny'],lsoa['minx'],lsoa['maxx'],network_type = 'bike',retain_all=True,simplify=False)
/home/chris/miniconda3/envs/gt/lib/python3.12/site-packages/shapely/geometry/polygon.py:91: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return [float(c) for c in o]
/home/chris/miniconda3/envs/gt/lib/python3.12/site-packages/OSM2AT/get_at_networks.py:232: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  edge_attributes['cent_x'] = edge_attributes['geometry'].centroid.x
/home/chris/miniconda3/envs/gt/lib/python3.12/site-packages/

2024-02-23 15:32:25 Projected GeoDataFrame to 'EPSG:32630 / WGS 84 / UTM zone 30N'
2024-02-23 15:32:25 Projected GeoDataFrame to 'EPSG:4326 / WGS 84'
2024-02-23 15:32:25 Projected GeoDataFrame to 'EPSG:32630 / WGS 84 / UTM zone 30N'
2024-02-23 15:32:25 Projected GeoDataFrame to 'EPSG:4326 / WGS 84'
2024-02-23 15:32:25 Requesting data from API in 1 request(s)
2024-02-23 15:32:25 Resolved 'overpass-api.de' to '65.109.112.52'
2024-02-23 15:32:25 Pausing 0 second(s) before making HTTP POST request to 'overpass-api.de'
2024-02-23 15:32:25 Post https://overpass-api.de/api/interpreter?data=%5Bout%3Ajson%5D%5Btimeout%3A90%5D%5Bdate%3A%222016-06-01T01%3A00%3A00Z%22%5D%3B%28way%5B%22highway%22%5D%5B%22area%22%21~%22yes%22%5D%5B%22access%22%21~%22private%22%5D%5B%22highway%22%21~%22abandoned%7Cbus_guideway%7Cconstruction%7Ccorridor%7Celevator%7Cescalator%7Cfootway%7Cmotor%7Cno%7Cplanned%7Cplatform%7Cproposed%7Craceway%7Crazed%7Csteps%22%5D%5B%22bicycle%22%21~%22no%22%5D%5B%22service%22%21~%22priv

KeyboardInterrupt: 